# Optimizando el Flujo de Trabajo en Desarrollo y DevOps con Makim

![](https://raw.githubusercontent.com/osl-incubator/makim/main/artwork/v1/logo.png)

## Setup

In [1]:
# !pip install "makim==1.9.1"

## Makim

Problemas que intenta solucinar makim:

* integración con archivos de entorno
* usa un formato amigable (yaml) para la declaración de los targets (scripts) y sus dependencias.
* permite uso de argumentos
* documentación (help) como parte de la estructura
* permite integración con aplicaciones shells y otros lenguajes de programación
* agrupación por grupos y integración de targets de distintos grupos

In [2]:
%%writefile Makefile

ARG1:=
ARG2:=

target1:
        @echo "target1 $(ARG1)"

target2: target1
        @echo "target2 $(ARG2)"

Overwriting Makefile


In [3]:
!make target1

Makefile:6: *** missing separator (did you mean TAB instead of 8 spaces?).  Stop.


In [4]:
! sed -i 's/        /\t/g' Makefile

In [5]:
!make target1

target1 


In [6]:
!make target2

target1 
target2 


In [7]:
# oops no es ARGS1 es ARG1
!make target1 ARGS1=arg1

target1 


In [8]:
!make target1 ARG1=arg1

target1 arg1


In [9]:
# oops no es ARG1 es ARG2
!make target2 ARG1=arg1

target1 arg1
target2 


In [10]:
# let's check check the help
!make --help

Usage: make [options] [target] ...
Options:
  -b, -m                      Ignored for compatibility.
  -B, --always-make           Unconditionally make all targets.
  -C DIRECTORY, --directory=DIRECTORY
                              Change to DIRECTORY before doing anything.
  -d                          Print lots of debugging information.
  --debug[=FLAGS]             Print various types of debugging information.
  -e, --environment-overrides
                              Environment variables override makefiles.
  -E STRING, --eval=STRING    Evaluate STRING as a makefile statement.
  -f FILE, --file=FILE, --makefile=FILE
                              Read FILE as a makefile.
  -h, --help                  Print this message and exit.
  -i, --ignore-errors         Ignore errors from recipes.
  -I DIRECTORY, --include-dir=DIRECTORY
                              Search DIRECTORY for included makefiles.
  -j [N], --jobs[=N]          Allow N jobs at once; infinite jobs with no arg.
  -k, 

No hay ninguna información de nuestros targets :(

### Let's play with Makim!

In [11]:
%%writefile .makim.yaml
version: 1
groups:
  main:
    help: This is the main group, let's have fun
    targets:
      favorite-food:
        help: What is your favorite food?
        args:
          food:
            help: the name of the food
            type: string
            required: true
        run: |
          echo "my favorite food is {{ args.food }}"
      favorite-dessert:
        help: What is your favorite dessert?
        dependencies:
          - target: main.favorite-food
            args:
              food: pizza
          - target: main.favorite-food
            args:
              food: lasagna
        args:
          dessert:
            help: What is your favorite dessert?
            type: string
            required: false
            default: tiramisu
        run: |
          echo "my favorite dessert is {{ args.dessert }}"

Overwriting .makim.yaml


Problemas con el format yaml? 
Verifique tu archivo con https://www.yamllint.com/

**Cuáles son los targets disponibles?**

In [12]:
!makim --help

usage: Makim [--help] [--version] [--verbose] [--dry-run]
             [--makim-file MAKIM_FILE]
             [target]

Makim is a tool that helps you to organize and simplify your helper commands.

positional arguments:
  target
    Specify the target command to be performed. Options are:
    
    main:
    -----
      main.favorite-food => What is your favorite food?
        ARGS:
          --food: (string) the name of the food
      main.favorite-dessert => What is your favorite dessert?
        ARGS:
          --dessert: (string) What is your favorite dessert?

options:
  --help, -h
    Show the help menu
  --version
    Show the version of the installed Makim tool.
  --verbose
    Show the commands to be executed.
  --dry-run
    Show the commands but don't execute them.
  --makim-file MAKIM_FILE
    Specify a custom location for the makim file.

If you have any problem, open an issue at: https://github.com/osl-incubator/makim


In [13]:
!makim main.favorite-food

 [EE] The argument `food` is set as required. Please, provide that argument to proceed. 


In [14]:
!makim main.favorite-food --food quesumacha

my favorite food is quesumacha


In [15]:
!makim main.favorite-dessert

my favorite food is pizza
my favorite food is lasagna
my favorite dessert is tiramisu


In [16]:
!makim main.favorite-dessert --dessert "ice cream"

my favorite food is pizza
my favorite food is lasagna
my favorite dessert is ice cream


### Features

Let's have fun!

In [17]:
%%writefile .env
ENV=dev
SERVICES=s1,s2,s3

Overwriting .env


In [18]:
%%writefile docker-compose.yaml
version: "3.9"
services:
  s1:
    image: python:3.11
    command: "python -m http.server 18000"
  s2:
    image: python:3.11
    command: "python -m http.server 18001"
  s3:
    image: python:3.11
    command: "python -m http.server 18002"

Overwriting docker-compose.yaml


NOTA: necesitamos instalar docker-compose. Para ello, instalamos el paquete compose-go que es un wrapper para docker-compose 2 escrito go.

In [19]:
# !pip install compose-go

In [20]:
%%writefile .makim.yaml
version: 1

groups:
  docker:
    help: control the docker from makim
    targets:
      up:
        help: Start the services
        run: |
          docker compose up -d
      stop:
        help: Stop the services
        run: |
          docker compose stop

      config-with-bash:
        help: show information for the services using bash and using jinja2
        env-file:
          .env
        shell: bash
        run: |
          {% for service in env.SERVICES.split(",") %}
          docker compose config "{{service}}"
          {% endfor %}

      config-with-xonsh:
        help: show information for the services using xonsh (https://xon.sh/bash_to_xsh.html?highlight=env)
        env-file:
          .env
        run: |
          for service in {{ env.SERVICES.split(",") }}:
            docker compose config @(service)

  path:
    help: targets that handles path
    targets:
      workdir:
        help: shows how to use the working directory
        working-directory: /tmp
        run:
          pwd

  python:
    help: "some commands using python as interpreter"
    shell: python
    targets:
      test-a-gt-b:
        help: target that shows how to use assert
        args:
          a:
            help: Argument a
            type: int
            required: true
          b:
            help: Argument b
            type: int
            required: true
        run: assert {{ args.a }} > {{ args.b }}
      debug:
        help: target that shows how to use assert
        run: |
          breakpoint()
          print("breakpoint works!")
              

Overwriting .makim.yaml


In [21]:
!makim --help

usage: Makim [--help] [--version] [--verbose] [--dry-run]
             [--makim-file MAKIM_FILE]
             [target]

Makim is a tool that helps you to organize and simplify your helper commands.

positional arguments:
  target
    Specify the target command to be performed. Options are:
    
    docker:
    -------
      docker.up => Start the services
      docker.stop => Stop the services
      docker.config-with-bash => show information for the services using bash and using jinja2
      docker.config-with-xonsh => show information for the services using xonsh (https://xon.sh/bash_to_xsh.html?highlight=env)
    
    path:
    -----
      path.workdir => shows how to use the working directory
    
    python:
    -------
      python.test-a-gt-b => target that shows how to use assert
        ARGS:
          --a: (int) Argument a
          --b: (int) Argument b
      python.debug => target that shows how to use assert

options:
  --help, -h
    Show the help menu
  --version
    Sho

In [22]:
!makim docker.up

[+] Running 3/0
 ✔ Container pyconbolivia2023-s1-1  R...                                   0.0s 
 ✔ Container pyconbolivia2023-s2-1  R...                                   0.0s 
 ✔ Container pyconbolivia2023-s3-1  R...                                   0.0s 


In [23]:
!makim docker.config-with-bash

name: pyconbolivia2023
services:
  s1:
    command:
    - python
    - -m
    - http.server
    - "18000"
    image: python:3.11
    networks:
      default: null
networks:
  default:
    name: pyconbolivia2023_default
name: pyconbolivia2023
services:
  s2:
    command:
    - python
    - -m
    - http.server
    - "18001"
    image: python:3.11
    networks:
      default: null
networks:
  default:
    name: pyconbolivia2023_default
name: pyconbolivia2023
services:
  s3:
    command:
    - python
    - -m
    - http.server
    - "18002"
    image: python:3.11
    networks:
      default: null
networks:
  default:
    name: pyconbolivia2023_default


In [24]:
!makim docker.config-with-xonsh

name: pyconbolivia2023
services:
  s1:
    command:
    - python
    - -m
    - http.server
    - "18000"
    image: python:3.11
    networks:
      default: null
networks:
  default:
    name: pyconbolivia2023_default
name: pyconbolivia2023
services:
  s2:
    command:
    - python
    - -m
    - http.server
    - "18001"
    image: python:3.11
    networks:
      default: null
networks:
  default:
    name: pyconbolivia2023_default
name: pyconbolivia2023
services:
  s3:
    command:
    - python
    - -m
    - http.server
    - "18002"
    image: python:3.11
    networks:
      default: null
networks:
  default:
    name: pyconbolivia2023_default


In [25]:
!makim path.workdir

/tmp


In [26]:
!makim python.test-a-gt-b

 [EE] The argument `a` is set as required. Please, provide that argument to proceed. 


In [27]:
!makim python.test-a-gt-b --a 2 --b 3

Traceback (most recent call last):
  File "/tmp/tmp7bl1c4__.makim", line 1, in <module>
    assert 2 > 3
           ^^^^^
AssertionError
 

  RAN: /home/xmn/miniforge3/envs/pyconbo2023/bin/python /tmp/tmp7bl1c4__.makim

  STDOUT:


  STDERR:
 


In [28]:
!makim python.test-a-gt-b --a 2 --b 1

In [29]:
!pwd

/mnt/sda1/storage/dev/opensciencelabs/main/talks/pyconbolivia2023


In [30]:
# %%debug
# makim python.debug